In [4]:
import openai
from dotenv import load_dotenv, find_dotenv
import os
import sys

sys.path.append("../..")


_ = load_dotenv(find_dotenv())  # read local .env file

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

openai.api_key = os.environ["OPENAI_API_KEY"]

In [16]:
import langchain

langchain.debug = False

In [6]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    api_key=HUGGINGFACEHUB_API_TOKEN,
)

In [7]:
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup
import nest_asyncio
page = None
async def main(url):
    global page
    browser = await launch()
    page = await browser.newPage()
    await page.goto(url)
    
    # Get the page content after JavaScript execution
    page_content = await page.content()

    # Close the browser
    await browser.close()

    # Now you can use BeautifulSoup to parse the page content
    soup = BeautifulSoup(page_content, 'html.parser')

    # Perform scraping using BeautifulSoup on the dynamically rendered content
    # ...return soup
    # print(soup.prettify())
    return soup

nest_asyncio.apply()
url = "https://www.mdcsoftware.com.vn/"
soup = asyncio.get_event_loop().run_until_complete(main(url))



In [8]:
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
    ("h5", "Header 5"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text(soup.text[:12] + soup.text[58:])

chunk_size = 200
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

splits = text_splitter.split_documents(html_header_splits)

In [10]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS

db = await FAISS.afrom_documents(splits, embeddings)


In [14]:
from langchain.chat_models import ChatOpenAI
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
    RunnableParallel,
)
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

retriever = db.as_retriever()

template = """You're helpful assistant, please answer the question based only on the following context:
{context}
Your response based on user's question language.
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0,
    callbacks=[StreamingStdOutCallbackHandler()],
    streaming=True,
)
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

/home/codespace/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [17]:
chain.invoke("What is MDC Software?")

MDC Software is a company that specializes in combining strategy, technology, data science, and design to solve complex business challenges. They are committed to building high-quality web applications, utilizing artificial intelligence, internet of things, and big data analysis, as well as providing professional UX design services. They also offer 24/7 customer support and are located in Hanoi City.

'MDC Software is a company that specializes in combining strategy, technology, data science, and design to solve complex business challenges. They are committed to building high-quality web applications, utilizing artificial intelligence, internet of things, and big data analysis, as well as providing professional UX design services. They also offer 24/7 customer support and are located in Hanoi City.'